# Libraries

In [4]:
import json

In [5]:
import pandas as pd

In [6]:
from fuzzywuzzy import fuzz, process
import pprint

# Read in 2WikiMultihopQA

In [5]:
train_df = pd.read_json('data/2WikiMultihopQA/train.json')
train_df

,_id,type,question,context,supporting_facts,evidences,answer
0,13f5ad2c088c11ebbd6fac1f6bf848b6,bridge_comparison,Are director of film Move (1970 Film) and dire...,"[[Stuart Rosenberg, [Stuart Rosenberg (August ...","[[Move (1970 film), 0], [Méditerranée (1963 fi...","[[Move (1970 film), director, Stuart Rosenberg...",no
1,3057c6c4086111ebbd5dac1f6bf848b6,bridge_comparison,Do both films The Falcon (Film) and Valentin T...,"[[The Falcon Takes Over, [The Falcon Takes Ove...","[[The Falcon (film), 0], [Valentin the Good, 0...","[[The Falcon (film), director, Vatroslav Mimic...",no
2,89bc944808a111ebbd79ac1f6bf848b6,bridge_comparison,"Which film whose director is younger, Charge I...","[[Danger: Diabolik, [Danger:, Diabolik is a 1...","[[Charge It to Me, 1], [Danger: Diabolik, 1], ...","[[Charge It to Me, director, Roy William Neill...",Danger: Diabolik
3,633f80660bdd11eba7f7acde48001122,compositional,What is the date of birth of Mina Gerhardsen's...,"[[Pamela Jain, [Pamela Jain is an Indian playb...","[[Mina Gerhardsen, 1], [Rune Gerhardsen, 0]]","[[Mina Gerhardsen, father, Rune Gerhardsen], [...",13 June 1946
4,2dc3f9740bda11eba7f7acde48001122,compositional,What nationality is the director of film Weddi...,"[[Weekend in Paradise (1931 film), [Weekend in...","[[Wedding Night in Paradise (1950 film), 0], [...","[[Wedding Night in Paradise, director, Géza vo...",Hungarian
...,...,...,...,...,...,...,...
167449,56100d300bdc11eba7f7acde48001122,compositional,What is the place of birth of the director of ...,"[[S. N. Mathur, [S.N. Mathur was the Director ...","[[Rolling in Money, 0], [Albert Parker (direct...","[[Rolling in Money, director, Albert Parker], ...",New York
167450,3df1a97108ad11ebbd83ac1f6bf848b6,comparison,"Who was born first, Dušan Ninić or Eszter Balint?","[[Tom Dickinson, [Thomas Eastwood Dickinson( 1...","[[Dušan Ninić, 0], [Eszter Balint, 0]]","[[Dušan Ninić, date of birth, September 6, 195...",Dušan Ninić
167451,8be4ef3e0bdc11eba7f7acde48001122,compositional,When did the director of film Morchha die?,"[[Thomas Scott (diver), [Thomas Scott( 1907- d...","[[Morchha, 0], [Ravikant Nagaich, 0]]","[[Morchha, director, Ravikant Nagaich], [Ravik...",6 January 1991
167452,12357df20bdc11eba7f7acde48001122,compositional,What is the date of birth of the director of f...,"[[Peter Levin, [Peter Levin is an American dir...","[[Double Cross (1951 film), 0], [Riccardo Fred...","[[Double Cross, director, Riccardo Freda], [Ri...",24 February 1909


In [4]:
dev_df = pd.read_json('data/2WikiMultihopQA/dev.json')
dev_df

,_id,type,question,context,supporting_facts,evidences,answer
0,8813f87c0bdd11eba7f7acde48001122,compositional,Who is the mother of the director of film Poli...,"[[Maheen Khan, [Maheen Khan is a Pakistani fas...","[[Polish-Russian War (film), 1], [Xawery Żuław...","[[Polish-Russian War, director, Xawery Żuławsk...",Małgorzata Braunek
1,61a46987092f11ebbdaeac1f6bf848b6,comparison,"Which film came out first, Blind Shaft or The ...","[[Blind Shaft, [Blind Shaft is a 2003 film abo...","[[Blind Shaft, 0], [The Mask of Fu Manchu, 0]]","[[Blind Shaft, publication date, 2003], [The M...",The Mask Of Fu Manchu
2,e2a3bf2a0bdd11eba7f7acde48001122,compositional,"When did John V, Prince Of Anhalt-Zerbst's fat...","[[Waldemar I, Prince of Anhalt-Zerbst, [Waldem...","[[John V, Prince of Anhalt-Zerbst, 2], [Ernest...","[[John V of Anhalt-Zerbst, father, Ernest I, P...",12 June 1516
3,0cd3bdea0bde11eba7f7acde48001122,compositional,What is the award that the director of film We...,"[[Michael Govan, [Michael Govan( born 1963) is...",[[Wearing Velvet Slippers under a Golden Umbre...,[[Wearing Velvet Slippers under a Golden Umbre...,Myanmar Motion Picture Academy Awards
4,f9dcb4a60bda11eba7f7acde48001122,compositional,Where was the director of film Ronnie Rocket b...,"[[Jason Moore (director), [Jason Moore( born O...","[[Ronnie Rocket, 0], [David Lynch, 4]]","[[Ronnie Rocket, director, David Lynch], [Davi...","Missoula, Montana"
...,...,...,...,...,...,...,...
12571,523dfd220bda11eba7f7acde48001122,compositional,What is the place of birth of the director of ...,"[[Brian Kennedy (gallery director), [Brian Pat...","[[Beowulf & Grendel, 0], [Sturla Gunnarsson, 1]]","[[Beowulf & Grendel, director, Sturla Gunnarss...",Reykjavík
12572,50a052e1085511ebbd59ac1f6bf848b6,comparison,"Who is younger, Paul Mokeski or Vincent Cobos?","[[Alan Chipp, [Alan Chipp( born 9 January 1937...","[[Paul Mokeski, 0], [Vincent Cobos, 0]]","[[Paul Mokeski, date of birth, January 3, 1957...",Vincent Cobos
12573,abebf78408c611ebbd90ac1f6bf848b6,comparison,Were Massimo Bonanni and Gino Pasqualotto from...,"[[Willem Brakman, [Willem Pieter Jacobus Brakm...","[[Massimo Bonanni, 0], [Gino Pasqualotto, 0]]","[[Massimo Bonanni, country of citizenship, Ita...",yes
12574,f5175c840bdd11eba7f7acde48001122,compositional,Where was the place of burial of Sultan Cem's ...,"[[Muazzez Sultan, [Muazzez Sultan( died 13 Sep...","[[Sultan Cem, 1], [Mehmed the Conqueror, 9]]","[[Sultan Cem, father, Mehmed II], [Mehmed II, ...",Fatih Mosque


In [5]:
example = dev_df[dev_df['answer'] == 'Bhoomiyude Avakashikal']
example

,_id,type,question,context,supporting_facts,evidences,answer
32,1a114c5e088f11ebbd71ac1f6bf848b6,bridge_comparison,"Which film has the director born later, Bhoomi...","[[Jacques Décombe, [Jacques Décombe is a Frenc...","[[Bhoomiyude Avakashikal, 1], [The Majesty of ...","[[Bhoomiyude Avakashikal, director, T. V. Chan...",Bhoomiyude Avakashikal


In [6]:
print(example['context'].to_list())
print(example['supporting_facts'].to_list())
print(example['evidences'].to_list())

[[['Jacques Décombe', ['Jacques Décombe is a French author, actor and director born in 1953.']], ['The Majesty of the Law', ['The Majesty of the Law is a 1915 American drama silent film written and directed by Julia Crawford Ivers.', 'The film stars George Fawcett( in his film debut), Jane Wolfe, William Desmond, Myrtle Stedman, John Oaker and Charlie Ruggles.', 'The film was released on August 26, 1915, by Paramount Pictures.']], ['Julia Crawford Ivers', ['Julia Crawford Ivers( October 3, 1869 – May 8, 1930) was an American motion picture pioneer.']], ['T. V. Chandran', ['T. V. Chandran( born 23 November 1950) is an Indian film director, screenwriter, and actor, predominantly working in Malayalam cinema.', 'Born in a Malayali family in Telicherry, Chandran worked as a Reserve Bank of India employee before entering into the film industry.', 'He started his film career as an assistant director to P. A. Backer.', 'He also acted the lead role in Backer\'s highly acclaimed political drama"

# Read In Hot Pot QA

In [7]:
# Opening JSON file
f = open('data/Super-NaturalInstructions/HotPotQA/hotpot_train_v1.1.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
pprint.pprint(data[:1])
 
# Closing file
f.close()

[{'_id': '5a7a06935542990198eaf050',
  'answer': "Arthur's Magazine",
  'context': [['Radio City (Indian radio station)',
               ["Radio City is India's first private FM radio station and was "
                'started on 3 July 2001.',
                ' It broadcasts on 91.1 (earlier 91.0 in most cities) '
                'megahertz from Mumbai (where it was started in 2004), '
                'Bengaluru (started first in 2001), Lucknow and New Delhi '
                '(since 2003).',
                ' It plays Hindi, English and regional songs.',
                ' It was launched in Hyderabad in March 2006, in Chennai on 7 '
                'July 2006 and in Visakhapatnam October 2007.',
                ' Radio City recently forayed into New Media in May 2008 with '
                'the launch of a music portal - PlanetRadiocity.com that '
                'offers music related news, videos, songs, and other '
                'music-related features.',
                ' The Ra

In [7]:
hot_pot_df = pd.read_json('data/Super-NaturalInstructions/HotPotQA/hotpot_train_v1.1.json')
hot_pot_df

,supporting_facts,level,question,context,answer,_id,type
0,"[[Arthur's Magazine, 0], [First for Women, 0]]",medium,Which magazine was started first Arthur's Maga...,"[[Radio City (Indian radio station), [Radio Ci...",Arthur's Magazine,5a7a06935542990198eaf050,comparison
1,"[[Oberoi family, 0], [The Oberoi Group, 0]]",medium,The Oberoi family is part of a hotel company t...,"[[Ritz-Carlton Jakarta, [The Ritz-Carlton Jaka...",Delhi,5a879ab05542996e4f30887e,bridge
2,"[[Allie Goertz, 0], [Allie Goertz, 1], [Allie ...",hard,Musician and satirist Allie Goertz wrote a son...,"[[Lisa Simpson, [Lisa Marie Simpson is a ficti...",President Richard Nixon,5a8d7341554299441c6b9fe5,bridge
3,"[[Peggy Seeger, 0], [Peggy Seeger, 1], [Ewan M...",medium,What nationality was James Henry Miller's wife?,"[[Moloch: or, This Gentile World, [Moloch: or,...",American,5a82171f5542990a1d231f4a,bridge
4,"[[Cadmium chloride, 1], [Ethanol, 0]]",medium,Cadmium Chloride is slightly soluble in this c...,"[[Cadmium chloride, [Cadmium chloride is a whi...",alcohol,5a84dd955542997b5ce3ff79,bridge
...,...,...,...,...,...,...,...
90442,"[[Kerry Remsen, 1], [Bert Remsen, 0]]",medium,Kerry Remsen is the daughter of an actor with ...,"[[Kerry Remsen, [Kerry Remsen is an American a...",American,5a8f8db25542997ba9cb32b9,bridge
90443,"[[Northshore Mall, 0], [Northshore Mall, 4], [...",easy,"Who manages both Northshore Mall in Peabody, M...","[[Green Tree Mall, [Green Tree Mall is a shopp...",Simon Property Group,5ae4f3615542993aec5ec0fd,bridge
90444,"[[Charlee Johnson, 4], [DreamWorks, 0]]",medium,Charlee Johnson was part of a band that signed...,"[[Simon M. Woods, [Simon M. Woods is a British...",Amblin Partners,5a903fc95542990a984935bd,bridge
90445,"[[Salt to the Sea, 1], [MV Wilhelm Gustloff, 0]]",medium,What is the ship that sank in the Baltic sea a...,[[The I.V. Stalin White Sea – Baltic Sea Canal...,"MV ""Wilhelm Gustloff",5ab56e71554299494045efc8,bridge


check if this is the same dataset as 2wikimultihopqa

In [8]:
hot_pot_df['level'].value_counts()

level
medium    56814
easy      17972
hard      15661
Name: count, dtype: int64

In [9]:
hot_pot_df['type'].value_counts()

type
bridge        72991
comparison    17456
Name: count, dtype: int64

In [14]:
print(hot_pot_df.head(1))
print(hot_pot_df.head(1)['supporting_facts'])
print(hot_pot_df.head(1)['context'].values[0])

                                 supporting_facts   level  \
0  [[Arthur's Magazine, 0], [First for Women, 0]]  medium   

                                            question  \
0  Which magazine was started first Arthur's Maga...   

                                             context             answer  \
0  [[Radio City (Indian radio station), [Radio Ci...  Arthur's Magazine   

                        _id        type  
0  5a7a06935542990198eaf050  comparison  
0    [[Arthur's Magazine, 0], [First for Women, 0]]
Name: supporting_facts, dtype: object
[['Radio City (Indian radio station)', ["Radio City is India's first private FM radio station and was started on 3 July 2001.", ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).', ' It plays Hindi, English and regional songs.', ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Vis

# compare answers and questions

## Exact Matches

In [13]:
train_answers = set(train_df['answer'].to_list())
hot_pot_answers = set(hot_pot_df['answer'].to_list())
len(train_answers.intersection(hot_pot_answers))

1859

In [18]:
intersected_answers = train_answers.intersection(hot_pot_answers)
intersected_answers_questions = []

i = 0
for answer in intersected_answers:    
    if i > 10:
        break
    i = i + 1
    train_question = train_df['question'][train_df['answer'] == answer].values[0]
    hot_pot_question = hot_pot_df['question'][hot_pot_df['answer'] == answer].values[0]
    intersected_answers_questions.append((train_question, hot_pot_question, answer))

intersected_answers_questions

[('What is the place of birth of the director of film Addicted To Fame?',
  'What English county does the River Gwash flow through and the village of Litter Casterton reside in?',
  'Rutland'),
 ("Which award the director of film Life'S Whirlpool got?",
  'What award did Gene Hackman win for his performance in a movie that was adapted and fictionalized from a 1969 book by Robin Moore?',
  'Academy Award for Best Actor'),
 ('Which country the director of film Jaque Mate (Film) is from?',
  'The origin of the genre of music that Jaime Enrique Amara primarily signs was in which country?',
  'Dominican Republic'),
 ("When is Sacha Hamilton, Duchess Of Abercorn's husband's birthday?",
  'When was the husband of Alexandra Anastasia Hamilton, Duchess of Abercorn born?',
  '4 July 1934'),
 ('What is the place of birth of the composer of film Follow Me! (1972 Film)?',
  'The York River is a 13 mile stream in southeast Maine that flows southeast to the Atlantic Ocean at what town, which is home 

they seem different enough

In [21]:
train_questions = set(train_df['question'].to_list())
hot_pot_questions = set(hot_pot_df['question'].to_list())
len(train_questions.intersection(hot_pot_questions))

0

## Fuzzy Matching

In [ ]:
matched_questions = []
for question in train_questions:
    matched_questions.append((question, process.extract(question, hot_pot_questions, limit=5)))
matched_questions

In [ ]:
matched_questions.to_csv('data/Super-NaturalInstructions/HotPotQA/matched_questions.csv')

In [ ]:
def fuzzy_match_files(set1, set2, threshold=80):
    matched_pairs = []

    for file1 in set1:
        for file2 in set2:
            similarity_score = fuzz.ratio(file1, file2)
            if similarity_score >= threshold:
                matched_pairs.append((file1, file2, similarity_score))

    return matched_pairs

In [ ]:
matched_questions_2 = fuzzy_match_files(train_questions, hot_pot_questions)
matched_questions_2

In [ ]:
matched_questions_2.to_csv('data/Super-NaturalInstructions/HotPotQA/matched_questions_2.csv')

# Looking at other datasets

In [1]:
import json

## task028 drop answer generation

In [18]:
# Opening JSON file
f = open('data/Super-NaturalInstructions/task028_drop_answer_generation.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
pprint.pprint(data['Instances'][:5])
 
# Closing file
f.close()

# task_df = pd.DataFrame.from_dict(
# print(task_df.shape)
# head(task_df)

[{'id': 'task028-39dd0c05643d460a898642ba26dcb360',
  'input': 'Passage: Hoping to rebound from their loss to the Patriots, the '
           'Raiders stayed at home for a Week 16 duel with the Houston '
           'Texans.  Oakland would get the early lead in the first quarter as '
           'quarterback JaMarcus Russell completed a 20-yard touchdown pass to '
           'rookie wide receiver Chaz Schilens.  The Texans would respond with '
           'fullback Vonta Leach getting a 1-yard touchdown run, yet the '
           'Raiders would answer with kicker Sebastian Janikowski getting a '
           '33-yard and a 30-yard field goal.  Houston would tie the game in '
           'the second quarter with kicker Kris Brown getting a 53-yard and a '
           '24-yard field goal. Oakland would take the lead in the third '
           'quarter with wide receiver Johnnie Lee Higgins catching a 29-yard '
           'touchdown pass from Russell, followed up by an 80-yard punt return '
       

## task176 break decompose questions

In [19]:
# Opening JSON file
f = open('data/Super-NaturalInstructions/task176_break_decompose_questions.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
pprint.pprint(data['Instances'][:5])
 
# Closing file
f.close()

# task_df = pd.DataFrame.from_dict(
# print(task_df.shape)
# head(task_df)

[{'id': 'task176-dda76f429e084d789afc7b7bbf420b4d',
  'input': 'question: How many field goals were kicked during the first half?',
  'output': ['#1 return field goals \n'
             '#2 return #1 of the  first half \n'
             '#3 return number of  #2']},
 {'id': 'task176-7f0d38a86eb6488bb482662d14bc6f28',
  'input': 'question: If one person in a bathing suit is in a pool with a '
           'guinea pig in the right image, and the person is behind the guinea '
           'pig.',
  'output': ['#1 return right image \n'
             '#2 return person in   #1 \n'
             '#3 return #2 that is in   bathing suit \n'
             '#4 return pool in   #1 \n'
             '#5 return #3 that is in   #4 \n'
             '#6 return guinea pig in   #1 \n'
             '#7 return #5 that is with #6 \n'
             '#8 return if  #2 is behind #6 \n'
             '#9 return number of  #7 \n'
             '#10 return if  #9 is equal to  one \n'
             '#11 return if  both  #10 and 

In [8]:
len(data)

14

## task178 quartz question answering

In [20]:
# Opening JSON file
f = open('data/Super-NaturalInstructions/task178_quartz_question_answering.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
pprint.pprint(data['Instances'][:5])
 
# Closing file
f.close()

# task_df = pd.DataFrame.from_dict(
# print(task_df.shape)
# head(task_df)

[{'id': 'task178-a1b52d6711d2437e9c5bb2a9321001ab',
  'input': "Question: John's town used to have lots of water, back when there "
           'were only a few hundred people. However, now that the town holds '
           'several thousand people, the water availability is \n'
           ' Option1: scarce \n'
           ' Option2: plentiful \n'
           ' Explanation: Many of the worlds people live with water scarcity, '
           'and that percentage will increase as populations increase and '
           'climate changes.',
  'output': ['scarce']},
 {'id': 'task178-2ad87a9c819b46d7acd7b78a765f2d67',
  'input': 'Question: Electrons further away from a nucleus have _____ energy '
           'levels than close ones. \n'
           ' Option1: higher \n'
           ' Option2: lower \n'
           ' Explanation: Electrons at lower energy levels, which are closer '
           'to the nucleus, have less energy.',
  'output': ['higher']},
 {'id': 'task178-d5e9963418f04345b46affcf366baf96',


In [31]:
# if you get rid of the options then there are multiple ways to phrase the correct answer
# the format works pretty well
def parse_instance(instance):
    inputs = instance['input'].split('\n')
    inputs[0] = inputs[0].replace('Question: ', '')
    inputs[3] = inputs[3].replace('Explanation: ', '')

    evidences = inputs[3].split('.')
    if evidences[-1] == '':
        evidences = evidences[:-1]
        
    evidences.extend(inputs[0].split('.'))
    question = evidences[-1]
    evidences = evidences[:-1]
    
    answer = instance['output'][0]
    
    return {'evidences': evidences, 'question': question, 'answer': answer}

[parse_instance(instance) for instance in data['Instances'][:5]]

[{'evidences': [' Many of the worlds people live with water scarcity, and that percentage will increase as populations increase and climate changes',
   "John's town used to have lots of water, back when there were only a few hundred people"],
  'question': ' However, now that the town holds several thousand people, the water availability is ',
  'answer': 'scarce'},
 {'evidences': [' Electrons at lower energy levels, which are closer to the nucleus, have less energy',
   'Electrons further away from a nucleus have _____ energy levels than close ones'],
  'question': ' ',
  'answer': 'higher'},
 {'evidences': [' An object with greater mass or greater velocity has more kinetic energy',
   'Milo threw both a basketball and a baseball through the air'],
  'question': ' If the basketball has more mass then the baseball, which ball has more kinetic energy? ',
  'answer': 'basketball'},
 {'evidences': [' If there are more greenhouse gases in the atmosphere, more ultraviolet radiation will be

In [36]:
# Opening JSON file
data = []
with open('data/Super-NaturalInstructions/quartz-dataset-v1-aug2019/train.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    data.append(json.loads(json_str))

pprint.pprint(data[:5])

[{'answerKey': 'A',
  'id': 'QRQA-10385-4',
  'para': 'Many of the worlds people live with water scarcity, and that '
          'percentage will increase as populations increase and climate '
          'changes.',
  'para_anno': {'cause_dir_sign': 'MORE',
                'cause_dir_str': 'increase',
                'cause_prop': 'scarcity',
                'effect_dir_sign': 'MORE',
                'effect_dir_str': 'increase',
                'effect_prop': 'population growth'},
  'para_id': 'QRSent-10385',
  'question': {'choices': [{'label': 'A', 'text': 'scarce'},
                           {'label': 'B', 'text': 'plentiful'}],
               'stem': "John's town used to have lots of water, back when "
                       'there were only a few hundred people. However, now '
                       'that the town holds several thousand people, the water '
                       'availability is'},
  'question_anno': {'less_cause_dir': 'lots of',
                    'less_cause_pr

## task237 iirc answer from some text to answer generation

In [38]:
# i've read a few of the answers and the link information and it doesn't make sense.the answers don't appear in the question, not explicit reasoning
# but if you look at the original dataset that is not used and supernatural instructions it is more useful

In [40]:
# Opening JSON file
f = open('data/Super-NaturalInstructions/iirc_train_dev/train.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
pprint.pprint(data[:3])
 
# Closing file
f.close()

[{'links': [{'indices': [73, 84], 'target': 'Air Support'},
            {'indices': [89, 101], 'target': 'Interdiction'},
            {'indices': [125, 143], 'target': 'Operation Overlord'},
            {'indices': [161, 169], 'target': 'Normandy'},
            {'indices': [279, 284], 'target': 'Normandy landings'},
            {'indices': [345, 357], 'target': 'Heavy bomber'},
            {'indices': [411, 415], 'target': 'Caen'},
            {'indices': [420, 428], 'target': 'Saint-Lô'},
            {'indices': [501, 524], 'target': 'Operation Market Garden'},
            {'indices': [571, 576], 'target': 'Rhine'},
            {'indices': [627, 633], 'target': 'Allies of World War II'},
            {'indices': [646, 654], 'target': 'Nijmegen'},
            {'indices': [701, 720], 'target': 'Battle of the Bulge'},
            {'indices': [729, 746], 'target': 'Operation Varsity'},
            {'indices': [806, 811], 'target': 'Wesel'},
            {'indices': [880, 888], 'target': 'Sa

In [43]:
pprint.pprint(data[0]['questions'])

[{'answer': {'answer_spans': [{'end': 160,
                               'passage': 'operation market garden',
                               'start': 131,
                               'text': ' from 17 to 25 September 1944'}],
             'type': 'span'},
  'context': [{'indices': [494, 655],
               'passage': 'main',
               'text': 'During Operation Market Garden, the attempt to seize a '
                       'bridgehead across the Rhine in the Netherlands, the '
                       '704th dropped supplies to allied troops near '
                       'Nijmegen.'},
              {'indices': [0, 124],
               'passage': 'Operation Market Garden',
               'text': 'Operation Market Garden was a failed World War II '
                       'military operation fought in the Netherlands from 17 '
                       'to 25 September 1944.'}],
  'qid': 'q_0',
  'question': 'When did the operation during which the 704th dropped supplies '
          